<a href="https://colab.research.google.com/github/Ashish-Soni08/100-Days-Of-Code/blob/main/course/week2/DCDL_Week_2_Tensor_Playground_Ashish_Soni.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensor Playground

A quick bag of tips for working with Tensors. 

In [1]:
import torch
import torch.nn.functional as F

One of the best practices for DL programming is to annotate expected shape as you go. Here's an example:

In [2]:
x = torch.randn(2, 3)  # shape: 2 x 3
x = x[0] * x[1] # shape: 3
x = x.unsqueeze(1) # shape: 3 x 1

This makes it a lot easier for the reader to keep track of what is going on.

## Why is shape even important?

One big reason is that we often want to do vector and matrix multiplications in deep learning. If the shapes aren't exactly matching, then unexpected things might happen. Here are some examples:

In [3]:
# We expect a shape of (2, 1)
A = torch.randn(2, 3)  # shape: 2 x 3
B = torch.randn(3, 1)  # shape: 3 x 1
C = A @ B              # shape: 2 x 1
print(C.size())

torch.Size([2, 1])


In [4]:
# If we pass a vector as the second argument, it will not error out. 
# Rather it performs a matrix vector multiplication, returning a vector
A = torch.randn(2, 3)  # shape: 2 x 3
B = torch.randn(3)     # shape: 3
C = A @ B              # shape: 2
print(C.size())

torch.Size([2])


In [5]:
# If we pass a vector as the first argument, it also will not error out. 
# It will again do vector-matrix multiplication, returning a vector
A = torch.randn(3)     # shape: 3
B = torch.randn(3, 2)  # shape: 3 x 2
C = A @ B              # shape: 2
print(C.size())

torch.Size([2])


In [6]:
# These ideas translate to larger vectors of course
# Below, we basically "ignore" the first dimension of A and 
# perform a matrix-matrix multiplication (2,3) times (3,5) a total
# of 16 times, resulting in sixteen (2,5) matrices
A = torch.randn(16, 2, 3)   # shape: 16 x 2 x 3
B = torch.randn(3, 5)       # shape: 3 x 5
C = A @ B                   # shape: 16 x 2 x 5
C.size()

torch.Size([16, 2, 5])

In [7]:
# The first dimension often has the meaning of being batch size. 
# In these cases, we might want to do "batch matrix multiplications"
mb = 16
A = torch.randn(mb, 2, 3)  # shape: mb x 2 x 3
B = torch.randn(mb, 3, 5)  # shape: mb x 3 x 5
# `bmm` stands for batch matrix multiplication
# You can also use `torch.matmul` or `torch.einsum`.
C = torch.bmm(A, B)        # shape: mb x 2 x 5
C.size()  # this is different than above!

torch.Size([16, 2, 5])

The other reason is that library functions like objective functions expect very specific shapes. We can see documentation of this online but it is important to make sure shapes match. Otherwise, again unexpected things happen.

In [8]:
# here is a tricky one
x = torch.randn(3)
total = torch.sum(x)  # what shape is total?

In [9]:
total.size()  # blank!
print(total)

tensor(-1.2011)


This is a special torch Tensor. You might expect summing a vector to return a float, but this is not a Python float. It is a "0-dim" torch Tensor containing only one number. 

In [10]:
total = total.unsqueeze(0)  # for example we can add a dimension with unsqueeze
total.size()                # shape: 1

torch.Size([1])

A common case when we need to care about shape is for loss functions. For example, let's look at binary cross entropy and cross entropy. 

In [11]:
# Cross-entropy on a 3-way classification problem

num_class = 3
num_examples = 64

logits = torch.randn(num_examples, num_class)  # shape: 64 x 3
labels = torch.randint(0, 3, (num_examples,))  # shape: 64

loss = F.cross_entropy(logits, labels)         # shape: 0
print(loss)

tensor(1.3246)


In [12]:
# Binary cross-entropy on a 2-way classification problem

num_examples = 64

logits = torch.randn(num_examples, 1)                      # shape: 64 x 1
labels = torch.randint(0, 1, (num_examples,))              # shape: 64
# read the documentation! This loss function expects labels to be (N, 1) shape
# and expects it to be a FloatTensor, not a LongTensor. 
labels = labels.unsqueeze(1).float()                       # shape: 64 x 1
loss = F.binary_cross_entropy_with_logits(logits, labels)  # shape: 0
print(loss)

tensor(0.7597)


### Common Operations

Four functions you will use a lot in PyTorch are `squeeze`, `unsqueeze`, `repeat`, and `view`. These are quite universal functions. In other languages like Tensorflow or Matlab, these functions may not be called the same thing but the same functions exist. 

In [13]:
# `squeeze(dim)` removes a dimension at index `dim`. 
# But it can only do so if the dimension at `dim` is size 1. 
x = torch.randn(16, 5, 2, 3)  # shape: 16 x 5 x 2 x 3
print(x.squeeze(0).size())    # does nothing!

torch.Size([16, 5, 2, 3])


In [14]:
x = torch.randn(16, 1, 2, 3)  # shape: 16 x 1 x 2 x 3
print(x.squeeze(1).size())    # shape: 16 x 2 x 3

torch.Size([16, 2, 3])


In [15]:
# If you don't pass a `dim`, squeeze will get rid of all "useless" dimensions
x = torch.randn(16, 1, 1, 1)  # shape: 16 x 1 x 1 x 1
print(x.squeeze().size())     # shape: 16

torch.Size([16])


In [16]:
# `unsqueeze(dim)` does the opposite. It adds a dimension of 1 at a position `dim`
x = torch.randn(16, 5, 2, 3)  # shape: 16 x 5 x 2 x 3
print(x.unsqueeze(0).size())  # shape: 1 x 16 x 5 x 2 x 3
print(x.unsqueeze(1).size())  # shape: 16 x 1 x 5 x 2 x 3
print(x.unsqueeze(2).size())  # shape: 16 x 5 x 1 x 2 x 3
print(x.unsqueeze(3).size())  # shape: 16 x 5 x 2 x 1 x 3
print(x.unsqueeze(4).size())  # shape: 16 x 5 x 2 x 3 x 1

torch.Size([1, 16, 5, 2, 3])
torch.Size([16, 1, 5, 2, 3])
torch.Size([16, 5, 1, 2, 3])
torch.Size([16, 5, 2, 1, 3])
torch.Size([16, 5, 2, 3, 1])


In [17]:
# A common operation you might see is to unsqueeze and then repeat
x = torch.randn(16)                        # shape: 16
print(x.unsqueeze(1).repeat(1, 5).size())  # shape: 16 x 5

torch.Size([16, 5])


In [18]:
# Finally, a popular thing is to reshape tensor sizes. 
x = torch.randn(120)             # shape: 120
print(x.view(12, 2, 5).size())   # shape: 12 x 2 x 5

torch.Size([12, 2, 5])
